In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import requests
from PIL import Image
import io
import os
import torch
from tqdm import tqdm
from peft import PeftModel, PeftConfig
from transformers import AutoProcessor, AutoModelForPreTraining
import time
import requests
from datetime import datetime


#Set the batch-size
batch_size = 8
# We already divided the test-set into 4-parts. 
# !!! If you have a central name please update the path !!!
# 1. Path to update
data_dir = "/teamspace/studios/this_studio/images/" 
# 2. Path to update
metadata_df = pd.read_csv("/teamspace/studios/this_studio/test.csv")


device = "cuda"
model_id = "google/paligemma-3b-ft-docvqa-448"
config = PeftConfig.from_pretrained("adithyabalagoni11/pali-gemma-ft-ml-challenge")
processor = AutoProcessor.from_pretrained(model_id)
base_model = AutoModelForPreTraining.from_pretrained("google/paligemma-3b-ft-docvqa-448").to(device)
peft_model = PeftModel.from_pretrained(base_model, "adithyabalagoni11/pali-gemma-ft-ml-challenge").to(device)
base_model.eval()
print("Locked and loaded!")





`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Locked and loaded!


In [30]:
pred_vals = {"predictions":[], "img_name": [],'labels':[]}
# Prepare batch inference
import random
error_files = ""
def extract_value(response):
    try:
        return response.split('\n')[-1].strip()
    except IndexError:
        return response  
#The for-loop was used for batch-inference as using test-loaders was giving errors
batch_size=8
for batch_start in tqdm(range(0, len(metadata_df), batch_size)):
    batch_end = min(batch_start + batch_size, len(metadata_df))

    #Updating the batch
    batch_images = []
    batch_prompts = []
    for idx in range(batch_start, batch_end):
        img_path =  data_dir +  metadata_df["image"][idx]
        pred_vals["img_name"].append(metadata_df["image"][idx])
        name=metadata_df["entity_name"][idx]
        parsed=name.split(' ')[3].replace('?','')
        forma=f"""" the {name} is  """
        prompt =name +forma

        # Using img_path to load the image
        image = Image.open(img_path).convert('RGB')

        # (IMG,PROMPT) batch
        batch_images.append(image)
        batch_prompts.append(prompt)

    #Pre-processing the batch
    model_inputs = processor(text=batch_prompts, images=batch_images, return_tensors="pt", padding=True).to(device)
    
    with torch.inference_mode():
        generation = base_model.generate(**model_inputs, max_new_tokens=100, do_sample=False)
        
    decoded = processor.batch_decode(generation, skip_special_tokens=True)
    # values=decoded
    # print(values)
    values = [extract_value(response) for response in decoded]
    # print(values)
    # print(random.sample(values,k=2))
    pred_vals["predictions"].extend(values)
    # if batch_start % 200 == 0 and batch_start > 0:
    #     pred_df = pd.DataFrame(pred_vals)
    #     pred_df.to_csv(f"test_set_{test_id}_pred_palli_final.csv", index=False)
pred_vals['labels']=metadata_df['entity_value'].tolist()





100%|██████████| 623/623 [1:09:01<00:00,  6.65s/it]


In [ ]:
pred_df = pd.DataFrame(pred_vals)
entity_unit_map = {
  "width": [
    "centimetre",
    "foot",
    "millimetre",
    "metre",
    "inch",
    "yard"
  ],
  "depth": ["centimetre",
    "foot",
    "millimetre",
    "metre",
    "inch",
    "yard"
  ],
  "height": [
    "centimetre",
    "foot",
    "millimetre",
    "metre",
    "inch",
    "yard"
  ],
  "item_weight": [
    "milligram",
    "kilogram",
    "microgram",
    "gram",
    "ounce",
    "ton",
    "pound"
  ],
  "maximum_weight_recommendation": [
    "milligram",
    "kilogram",
    "microgram",
    "gram",
    "ounce",
    "ton",
    "pound"
  ],
  "voltage": [
    "millivolt",
    "kilovolt",
    "volt"
  ],
  "wattage": [
    "kilowatt",
    "watt"
  ],
  "item_volume": [
    "cubic foot",
    "microlitre",
    "cup",
    "fluid ounce",
    "centilitre",
    "imperial gallon",
    "pint",
    "decilitre",
    "litre",
    "millilitre",
    "quart",
    "cubic inch",
    "gallon"
  ]
}

def parse_measurement(text, entity_unit_map):
    """
    Parse measurement text to standardized 'value unit' format using only allowed units
    """
    if pd.isna(text):
        return None
        
    text = str(text).lower().strip()
    
    # Unit mapping dictionary
    unit_mapping = {
        # Length
        'cm': 'centimetre',
        'mm': 'millimetre',
        'm': 'metre',
        'in': 'inch',
        'inch': 'inch',
        '"': ' ',
        'inches': 'inch',
        'ft': 'foot',
        'feet': 'foot',
        'yd': 'yard',
        
        # Weight
        'g': 'gram',
        'grams': 'gram',
        'kg': 'kilogram',
        'mg': 'milligram',
        'µg': 'microgram',
        'oz': 'ounce',
        'lbs': 'pound',
        'lb': 'pound',
        
        # Volume
        'ml': 'millilitre',
        'l': 'litre',
        'cl': 'centilitre',
        'dl': 'decilitre',
        'fl oz': 'fluid ounce',
        'gal': 'gallon',
        'pt': 'pint',
        'qt': 'quart',
        'cu in': 'cubic inch',
        'cu ft': 'cubic foot',
        
        # Electrical
        'v': 'volt',
        'mv': 'millivolt',
        'kv': 'kilovolt',
        'w': 'watt',
        'kw': 'kilowatt'
    }
    
    # Clean text
    text = re.sub(r'\([^)]*\)', '', text)  # Remove parentheses content
    
    # Modified pattern to capture number followed by unit
    pattern = r'(\d*\.?\d+)\s*([a-zA-Z"\']+|(?:fl oz)|(?:cu ft)|(?:cu in))'
    matches = re.finditer(pattern, text)
    
    try:
        # Get first match
        match = next(matches)
        value = float(match.group(1))
        unit_text = match.group(2).strip().lower()
        
        # Get all valid units from entity_unit_map
        valid_units = set()
        for units in entity_unit_map.values():
            valid_units.update(units)
            
        # Try to find matching unit
        found_unit = None
        
        # First check unit_mapping
        if unit_text in unit_mapping and unit_mapping[unit_text] in valid_units:
            found_unit = unit_mapping[unit_text]
            
        # Then check valid_units directly
        if not found_unit:
            for valid_unit in valid_units:
                if valid_unit.lower().startswith(unit_text):
                    found_unit = valid_unit
                    break
        if value.is_integer():
            value=int(value)
        if found_unit:
            return f"{value} {found_unit}"
            
    except (StopIteration, ValueError):
        pass
        
    return ""

def process_predictions(df, prediction_col, entity_unit_map=entity_unit_map):
    """
    Process predictions column and create new standardized column
    """
    df['parsed_predictions'] = df[prediction_col].apply(
        lambda x: parse_measurement(x, entity_unit_map)
    )
    
    return df
pred_df=process_predictions(pred_df,'predictions')
pred_df.to_csv('test_pali-gemma-base.csv')

In [ ]:
pred_vals = {"predictions":[], "img_name": [],'labels':[]}
# Prepare batch inference
import random
def extract_value(response):
    try:
        return response.split('\n')[-1].strip()
    except IndexError:
        return response  
#The for-loop was used for batch-inference as using test-loaders was giving errors
batch_size=8
for batch_start in tqdm(range(0, len(metadata_df), batch_size)):
    batch_end = min(batch_start + batch_size, len(metadata_df))

    #Updating the batch
    batch_images = []
    batch_prompts = []
    for idx in range(batch_start, batch_end):
        img_path =  data_dir +  metadata_df["image"][idx]
        pred_vals["img_name"].append(metadata_df["image"][idx])
        name=metadata_df["entity_name"][idx]
        parsed=name.split(' ')[3].replace('?','')
        forma=f"""" the {name} is  """
        prompt =name +forma

        # Using img_path to load the image
        image = Image.open(img_path).convert('RGB')

        # (IMG,PROMPT) batch
        batch_images.append(image)
        batch_prompts.append(prompt)

    #Pre-processing the batch
    model_inputs = processor(text=batch_prompts, images=batch_images, return_tensors="pt", padding=True).to(device)
    
    with torch.inference_mode():
        generation = peft_model.generate(**model_inputs, max_new_tokens=100, do_sample=False)
        
    decoded = processor.batch_decode(generation, skip_special_tokens=True)
    # values=decoded
    # print(values)
    values = [extract_value(response) for response in decoded]
    # print(values)
    # print(random.sample(values,k=2))
    pred_vals["predictions"].extend(values)
    # if batch_start % 200 == 0 and batch_start > 0:
    #     pred_df = pd.DataFrame(pred_vals)
    #     pred_df.to_csv(f"test_set_{test_id}_pred_palli_final.csv", index=False)
pred_vals['labels']=metadata_df['entity_value'].tolist()





In [ ]:
pred_df = pd.DataFrame(pred_vals)
pred_df.to_csv("test_pali-gemma-ft-ml-challenge.csv", index=False)

In [70]:

def compute_metrics(csv,pred_column):
    df=pd.read_csv(csv)
    predictions,labels=df[pred_column].tolist(),df['labels'].tolist()

    true_positives = 0
    false_positives = 0
    false_negatives = 0
    true_negatives = 0
    
    # Calculate metrics based on the problem's criteria
    for pred, gt in zip(predictions, labels):
        if pred != "" and gt != "":
            if pred == gt:
                true_positives += 1
            else:
                false_positives += 1
        elif pred != "" and gt == "":
            false_positives += 1
        elif pred == "" and gt != "":
            false_negatives += 1
        else:  # pred == "" and gt == ""
            true_negatives += 1
    
    # Calculate precision and recall
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    
    # Calculate F1 score
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    accuracy=(true_positives+true_negatives)/(true_positives+true_negatives+false_positives+false_negatives)
    # Compile metrics
    metrics = {
        "f1_score": f1_score,
        "precision": precision,
        "recall": recall,
        "exact_match ( out of 5000 examples )": true_positives,
        'accuracy':accuracy 
    }
    for key in metrics:
        print(f"{key} : {metrics[key]} ")
    
    return metrics    


In [71]:
result=compute_metrics("test_pali-gemma-base.csv",'parsed_predictions')



f1_score : 0.5969854909142133 
precision : 0.4255020080321285 
recall : 1.0 
exact_match ( out of 5000 examples ) : 2119 
accuracy : 0.4255020080321285 


In [72]:
result=compute_metrics("test_pali-gemma-ft-ml-challenge.csv",'predictions')


f1_score : 0.7080036321183033 
precision : 0.547991967871486 
recall : 1.0 
exact_match ( out of 5000 examples ) : 2729 
accuracy : 0.547991967871486 
